# Lecture 5: Pandas

Pandas is a python library designed to make many common types of data analysis straightforward. It borrows heavily from R. 

### Memory structure

The basic datastructure in Pandas is a DataFrame. These data frames are very similar to Numpy Structured Arrays, except how they are stored in memory is completely different. 

Structured arrays consist of a Numpy array with an entry for each row in the structured array, and each entry is a tuple. 

Pandas data frames consist of a dictionary-like object (with keys indicating rows or indexes) containing a set of columns. Each column is stored as a Numpy 1D array (actually, Pandas has a wapper for the 1D array called a Series that shows up here and there) that consists of that array plus an additional array that contains indexes.

This hybrid representation makes computations within a column very fast, and computations across columns not particularly ineffient because of the extra index array stored with each column. The sacrifice in this case is storage space, Pandas dataframes take up more memory than Numpy or Scipy arrays.

### Advantages

Pandas is designed to a few things very well. It is incredibly useful for grouping data, aggregating and processing these groups, and reshaping data. Support for time-series analyses in Pandas is also strong with built-in date and time representations.

In [ ]:
import numpy
import pandas

## Reading and writing Pandas data frames

In [ ]:
# read a data file into pandas
data = pandas.read_csv("iris.txt", delim_whitespace=True, header=None)

In [ ]:
# save the datafile as a csv
data.to_csv("tmp.csv", sep=",")

## Getting some simple information

In [ ]:
# first 5 rows
print(data.head())
# last 5 rows
print(data.tail())

In [ ]:
data.columns = ["sepal_length", "sepal_width", "petal_length", "petal_width", "type"]
print(data.head(10))

In [ ]:
print(data.info())

In [ ]:
print(data.describe())

In [ ]:
# pandas has some nice default printing options when not passed through print
data.head()

## Indexing data frames

As with any data structure, it is important to be able to access subsets of the dataframe. Pandas has tools for accessing both rows and columns in a number of ways.

In [ ]:
# index rows with just a set of indices (no comma)
print(data[:5])
# data[:5,] # causes an error!

In [ ]:
# print the names of the columns
print(data.columns)
print(list(data))

In [ ]:
# single column (note that only the first 5 rows are selected):
print(data['sepal_width'][:5])

In [ ]:
# more than one column:
print(data[['sepal_width', 'petal_width']][:5])

In [ ]:
# boolean indexing works
print(data[(data['type'] == "virginica") & (data['petal_length'] < 5)])

In [ ]:
# rows can also have names:
summary = data.describe()
summary

In [ ]:
# and we can index rows using the loc property:
print(summary.loc['mean'])
print()
print(summary.loc[['mean', '50%']])
print()
print(summary.loc['25%':'75%'])

In [ ]:
# or by row index number using iloc:
print(summary.iloc[1:3])

## Reshaping and pivot tables

### Summarizing data

Perhaps one of the most useful way to summarize data and understand it is to simply count entries. Here we show a few exaples of recoding continuous values into a binary value and then calcuating table counts.

In [ ]:
# some summary functions like crosstab work with selected columns
pandas.crosstab(data['petal_width'] > 2, columns=data['type'])

In [ ]:
# rows can be nested hierarchically
p_w_2 = data['petal_width'] > 2
p_l_2 = data['petal_length'] > 2
pandas.crosstab([p_w_2, p_l_2], columns=data['type'])

In [ ]:
# columns can be nested hierarchically
pandas.crosstab(data['type'], [p_w_2, p_l_2], 
                colnames=["Width > 2", "Lenght > 2"], 
                rownames=["Flower Type"])

In [ ]:
# count up the values
data['type'].value_counts()

### Pivot tables

Pivot tables are a useful tool for summarizing data along different groupings. Pandas provides a wapper function called pivot_table for constructing these tables.

In [ ]:
example = pandas.DataFrame({'A': ['one', 'one', 'two', 'three'] * 6,
                       'B': ['X', 'Y', 'Z'] * 8,
                       'C': ['foo', 'foo', 'foo', 'bar', 'bar', 'bar'] * 4,
                       'D': numpy.random.randn(24),
                       'E': numpy.random.randn(24)})
example.head(10)

In [ ]:
# build a pivot table:
# columns parameter specifies the column types, index specifies the rows
pandas.pivot_table(example, values='D', index=['A', 'B'], columns=['C'])

In [ ]:
# more than one column in the dataframe can be used for a value
# and pandas will automatically guess which columns to use
pandas.pivot_table(example, index=['A', 'B'], columns=['C'])

In [ ]:
# more than one column can be specified
pandas.pivot_table(example, values=['D','E'], index=['B'], columns=['A', 'C'])

In [ ]:
# default aggregation is mean
# but we can specify other aggreation functions:
pandas.pivot_table(example, values='D', index=['B'], 
                   columns=['A', 'C'], aggfunc=numpy.sum)

In [ ]:
# pivot_table has many additional optional parameters but margins is a useful one:
# note that we are also using pivot_table as a method of the dataframe
example.pivot_table(index=['A', 'B'], columns='C', 
                    margins=True, aggfunc=numpy.median)

## Groupby

One of the most powerful features in pandas is the ability to group dataframes by specific values and then perform calculations on those groups. These features are directly stolen from how R does this.

In [ ]:
df = pandas.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar', 
                              'foo', 'bar', 'foo', 'foo'],
                       'B' : ['one', 'one', 'two', 'three', 
                              'two', 'two', 'one', 'three'],
                       'C' : numpy.random.randn(8),
                       'D' : numpy.random.randn(8)})
df

In [ ]:
# group by a single column
df.groupby('A').describe()

In [ ]:
# group by more than one column
df.groupby(['A', 'B']).describe()

In [ ]:
# by default, groupby sorts the groups (alphabetically or numerically)
print(df.groupby('B').mean())
print()
print(df.groupby('B', sort=False).mean())

### Accessing groups and iterating

The object created by the function groupy() is actually a list that can be iterated across to perform actions on each group of the data. Pandas provides some vectorized functions and others can be written:

In [ ]:
# can access a single group
print(df.groupby('A').get_group('bar'))

In [ ]:
# iterate through all groups
for name, group in df.groupby(['A', 'B']):
    print(name)
    print(group)

### Aggregation

Aggregation changes the number of rows or indices within the dataframe. We already saw one way to aggregate a group: describe(). There are many, many others:

- count
- cumcount
- first
- head
- last
- max
- mean
- median
- min
- nth
- prod
- size
- sem
- std
- sum
- var
- tail
- agg
- all
- any
- bfill
- corr
- count
- cov
- cummax
- cummin
- cumprod
- cumsum
- describe
- diff
- ffill
- fillna
- idxmax
- idxmin
- mad
- pct_change
- quantile
- rank
- resample
- shift
- size
- skew
- take
- tshift
- nlargest
- nsmallest
- value_counts
- corrwith


In [ ]:
# a few examples:
print(df.groupby('A').size())
print()
print(df.groupby('A').max())
print()
print(df.groupby('A').agg([numpy.sum, numpy.mean, numpy.std]))

## Concatinate, Join, and Merge

### Appending a row or column to a dataframe

In [ ]:
# append a row:
df = pandas.DataFrame(numpy.random.randn(8, 4), columns=['A','B','C','D'])
print(df)
s = df.iloc[3]
print(df.append(s, ignore_index=True))

In [ ]:
# append a column:
df['E'] = numpy.random.random(df.shape[0])
print(df)

In [ ]:
# delete that column
del df['E']
print(df.head())

### Concatinate data frames

In [ ]:
df1 = pandas.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                        'B': ['B0', 'B1', 'B2', 'B3'],
                        'C': ['C0', 'C1', 'C2', 'C3'],
                        'D': ['D0', 'D1', 'D2', 'D3']},
                       index=[0, 1, 2, 3])

df2 = pandas.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                        'B': ['B4', 'B5', 'B6', 'B7'],
                        'C': ['C4', 'C5', 'C6', 'C7'],
                        'D': ['D4', 'D5', 'D6', 'D7']},
                       index=[4, 5, 6, 7])

df3 = pandas.DataFrame({'A': ['A8', 'A9', 'A10', 'A11'],
                        'B': ['B8', 'B9', 'B10', 'B11'],
                        'C': ['C8', 'C9', 'C10', 'C11'],
                        'D': ['D8', 'D9', 'D10', 'D11']},
                       index=[8, 9, 10, 11])

df4 = pandas.DataFrame({'E': ['E2', 'E3', 'E6', 'E7'],
                        'F': ['F2', 'F3', 'F6', 'F7'],
                        'G': ['G2', 'G3', 'G6', 'G7']},
                       index=[0, 1, 2, 3])
print(df1)
print()
print(df2)
print()
print(df3)
print()
print(df4)

In [ ]:
# it is straightforward to build one dataframe
# note: this is numpy.concatinate under the hood so it is expensive in time and memory!
dfs = [df1, df2, df3]
pandas.concat(dfs)

In [ ]:
# more interestingly, we can add pandas indexes
result = pandas.concat(dfs, keys=['x', 'y', 'z'])
result

In [ ]:
# can then use these indexes to pull out individual dataframes:
result.loc['y']

### Specifying rows or columns for the concatination

In [ ]:
# pandas will default to looking to concatinate rows together
print(pandas.concat([df1, df4]))

In [ ]:
# you can specify which axis to merge along:

# default behavior
print(pandas.concat([df1, df4], axis=0))

# concatinate columns with the same index together!
print(pandas.concat([df1, df4], axis=1))


In [ ]:
# append can also be used but has fewer options:
df1.append([df2, df3])

In [ ]:
# if you are performing many concatinations, 
# it's much more efficient to use list comprehensions to do a single concatination:
if False:
    dfs = [ process_your_file(f) for f in files ]
    result = pandas.concat(dfs)

### Joins using the Concat function

Joining data frames is the process of combining two dataframes. This becomes interesting and complex when some (but not all) of the row indicies and column names overlap between the two dataframes.

In [ ]:
# df5 and df1 share some indicies and names:
df5 = pandas.DataFrame({'B': ['B2', 'B3', 'B6', 'B7'],
                        'D': ['D2', 'D3', 'D6', 'D7'],
                        'F': ['F2', 'F3', 'F6', 'F7']},
                       index=[2, 3, 6, 7])
print(df1)
print()
print(df5)

In [ ]:
# the default behavior of concat is to not duplicate column names 
# but to duplicate row indices
pandas.concat([df1, df5])

In [ ]:
# but the axis parameter can get concat to create duplicate 
# columns and only unique rows
pandas.concat([df1, df5], axis=1)

In [ ]:
# The default in this case is an outer join:
# all rows are retained regardless of each column is present in the row
# this can produce NaN values
pandas.concat([df1, df5], axis=1, join='outer')

In [ ]:
# can also specify an inner join:
# each row must have matching columns
# this can lead to dropping rows
pandas.concat([df1, df5], axis=1, join='inner')

In [ ]:
# concat can also be told to ignore duplicate indices
# this results in new index values being created for conflicts
print(df1)
print()
print(df5)
pandas.concat([df1, df5], ignore_index=True)

### Joins using the Join function

Join is a convenience function in pandas for joining two dataframes. It is a wrapper around the merge function.

In [ ]:
left = pandas.DataFrame({'A': ['A0', 'A1', 'A2'],
                         'B': ['B0', 'B1', 'B2']},
                        index=['K0', 'K1', 'K2'])
right = pandas.DataFrame({'C': ['C0', 'C2', 'C3'],
                          'D': ['D0', 'D2', 'D3']},
                         index=['K0', 'K2', 'K3'])
print(left)
print()
print(right)

In [ ]:
# default is a left join: 
# all indexes from the left dataframe are kept as well as 
# any indexes in the right that occur in the left
left.join(right)

In [ ]:
# we can also specify a right join
left.join(right, how="right")

In [ ]:
# or an outer join that keeps all indexes
left.join(right, how="outer")

In [ ]:
# or an inner join that keep all complete indexes
left.join(right, how="inner")

### Joining by key

Often datasets are related by a shared key. These are often referred to as relational datasets or databases. Pandas makes it easy to merge across shared key values.

In [ ]:
books = pandas.DataFrame({'title': ['Title1', 'Title2', 'Title3', 'Year4'],
                         'year': ['Year1', 'Year2', 'Year3', 'Year4'],
                         'key': ['A1', 'A2', 'A1', 'A2']})
authors = pandas.DataFrame({'first_name': ['F1', 'F2', 'F3'],
                          'last_name': ['L1', 'L2', 'L3']},
                          index=['A1', 'A2', 'A3'])
print(books)
print()
print(authors)

In [ ]:
books.join(authors, on="key")

### Joins using the merge function

Merge has more options than Join. Specifically, it is possible to specify a key in both dataframes to merge based on, to specify multiple keys within each dataframe, etc.

One really nice feature (that is new in 0.17 but not in the stable release on this server) is to include a new column indicating which dataframe the row came from.

In [ ]:
# currently broken but will be cool:
# pandas.merge(df1, df5, how='outer', indicator='indicator_column')

## Time series

Pandas has incredibly powerful tools for processing data based on time series.

In [ ]:
# create a range of dates
rng = pandas.date_range('1/1/2011', periods=24*8, freq='H')
print(rng.summary())

# a series of numbers with those dates as an index
ts = pandas.Series(numpy.random.randint(0, 500, len(rng)), index=rng)
print(ts.head(10))

In [ ]:
# convert time zone
ts_utc = ts.tz_localize('UTC')
print(ts_utc.head())

# convert to another time zone
print(ts_utc.tz_convert('US/Eastern').head())

### Transform the time increments

In [ ]:
# recode the frequency of the time steps
# fill in missing data from the previous step
print(ts.asfreq('45Min', method='pad').head(10))

In [ ]:
# bin the timestamps into days (from hours) and return the mean value 
print(ts.resample('D', how="mean"))

In [ ]:
# compute rolling means
lag = pandas.rolling_mean(ts, 5, min_periods=1)
print(lag.head(10))
print()

# compute rolling max
roll_max = pandas.rolling_max(ts, 3, min_periods=1)
print(roll_max.head(10))

# 10+ rolling functions in pandas 
# including a rolling_apply that takes an arbitrary function

### Converting from strings to timestamps

In [ ]:
# converting 
print(pandas.to_datetime(pandas.Series(['Jul 31, 2009', '2010-01-10', None])))
print()
print(pandas.to_datetime(['2005/11/23', '2010.12.31']))
print()

# European style dates:
print(pandas.to_datetime(['04-01-2012 10:00'], dayfirst=True))
print()
print(pandas.to_datetime(['14-01-2012', '01-14-2012'], dayfirst=True))
print()

# from epochs:
print(pandas.to_datetime([1349720105, 1349806505, 1349892905, 
                          1349979305, 1350065705], unit='s'))